In [13]:
%pylab inline
import cv2
print(cv2.__version__)

Populating the interactive namespace from numpy and matplotlib
4.1.1


# Haar Cascade Face Detection

In [16]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

detect_scale = float(2) #Scale used for Haar cascade detection
window_scale = float(2) #Scale of the interactive window

window_name = "Live Face Detection"

In [17]:
cv2.namedWindow(window_name)
vc = cv2.VideoCapture(0)

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    rval, frame = vc.read()
    
    #Preprocess image for detector
    frame_detect = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_detect = cv2.resize(frame_detect, (0, 0), fx=detect_scale, fy=detect_scale)
    
    #Detect faces
    faces_q = faceCascade.detectMultiScale(frame_detect, scaleFactor=1.2, minNeighbors=5, minSize=(5, 5), flags = cv2.CASCADE_SCALE_IMAGE)
    
    #Draw rectangles on detected faces
    for  (x, y, w, h) in faces_q:
        cv2.rectangle(frame, (int(x/detect_scale), int(y/detect_scale)), 
                      (int((x+w)/detect_scale), int((y+h)/detect_scale)), 
                      (0, 0, 255), 2)

    frame = np.flip(frame, 1) #Mirror image for webcam mirror use
    frame = cv2.resize(frame, (0, 0), fx=window_scale, fy=window_scale) #Scale the image to fit window size
    cv2.imshow(window_name, frame)
    
    key = cv2.waitKey(5)
    if key == 27: # exit on ESC
        break

cv2.destroyWindow(window_name)
vc.release()

# ORB Features Detection

In [14]:
MIN_MATCH_COUNT = 10

img1 = cv2.imread('tag-mosaic.png') # trainImage
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

detect_scale = float(1) #Scale used for ORB detection
window_scale = float(2) #Scale of the interactive window

window_name = "ORB Object Detection"


In [15]:
# Initiate ORB detector
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1,None)


FLANN_INDEX_LSH = 6
index_params= dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6, # 12
                   key_size = 12,     # 20
                   multi_probe_level = 1) #2

search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

cv2.namedWindow(window_name)
vc = cv2.VideoCapture(0)

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    rval, frame = vc.read()
    img2 = cv2.resize(frame, (0, 0), fx=detect_scale, fy=detect_scale)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    try:
        # find the keypoints and descriptors with ORB
        kp2, des2 = orb.detectAndCompute(img2,None)

        matches = flann.knnMatch(des1,des2,k=2)

        # Need to draw only good matches, so create a mask
        matchesMask = [[0,0] for i in range(len(matches))]

        # store all the good matches as per Lowe's ratio test.
        good = []
        if len(matches) > 0:
            for i, v in enumerate(matches):
                if len(v) == 2:
                    m, n = v
                    if m.distance < 0.7*n.distance:
                        good.append(m)
                        matchesMask[i]=[1,0]

        # if enough matches, compute homography
        if len(good)>MIN_MATCH_COUNT:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()

            h,w,d = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

            dst = cv2.perspectiveTransform(pts,M)
            img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
            
    except:
        error = 0


    frame = img2   
        
    frame = np.flip(frame, 1) #Mirror image for webcam mirror use
    frame = cv2.resize(frame, (0, 0), fx=window_scale, fy=window_scale) #Scale the image to fit window size
    cv2.imshow(window_name, frame)
    
    key = cv2.waitKey(5)
    if key == 27: # exit on ESC
        break

cv2.destroyWindow(window_name)
vc.release()